In [1]:
import sys
sys.path.insert(0, '..')

# Инициализируем граф

In [2]:
import pandas as pd

from src.eventstream.eventstream import Eventstream
from src.eventstream.schema import RawDataSchema, EventstreamSchema
from src.graph.p_graph import PGraph, EventsNode

raw_data = pd.read_csv('simple-onlineshop.csv')

raw_data_schema = RawDataSchema(
                            event_name="event",
                            event_timestamp="timestamp",
                            user_id="user_id")

source = Eventstream(
    raw_data=raw_data,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

graph = PGraph(source_stream=source)

In [3]:
source_df = source.to_dataframe()

## CutPathAfter

### Описание работы функции

Обрезает eventstream сразу после одного из событий, указанных в списке cutoff_events (первого в траектории). Остается часть траектории до первого найденного события из списка включительно. Само событие остается в траектории.

Параметры:
```cut_shift``` - смещение относительно обозначенного события в сторону сокращения траектории (влево)
```min_cjm``` - минимальное кол-во событий в траектории, которые надо оставить, если в траектории меньшее количество событий, то они будут удалены



### 1 событие

In [4]:
from src.data_processors_lib.rete import CutPathAfterEvent, CutPathAfterEventParams
params = {
    'cutoff_events': ['payment_done'],
    'cut_shift': 0,
    'min_cjm': 0
}

cut_after = EventsNode(
    CutPathAfterEvent(params=CutPathAfterEventParams(**params)))

graph.add_node(
    node=cut_after,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_after
)


/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_after_event.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut['rw_cumsum'] = df_cut.loc[::-1].groupby([user_col])[time_col].transform(
/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_after_event.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_del['ref'] = df_to_del[eventstream.schema.event_id]


In [5]:
df8 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df8.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [6]:
df8['event_type'].value_counts()

raw    32666
Name: event_type, dtype: int64

In [7]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [8]:
source_df[source_df['event_name'] == 'payment_done']['user_id'].nunique()

653

In [9]:
check_id = 629881394
print(len(df8[df8['user_id'] == check_id]))
event_time = source_df[(source_df['user_id'] == check_id) & \
    (source_df['event_name'] == 'payment_done')]['event_timestamp'].values[0]

26


In [10]:
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] <= event_time)]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,80f39ba7-6fe7-4807-b861-0bf310b0d3f8,raw,7,main,2019-11-01 22:28:54.791683,629881394
9,c96cfae1-7f00-49a1-a401-06a5ca509a9c,raw,9,catalog,2019-11-01 22:29:01.049513,629881394
11,897d59ef-4def-473d-b8b8-90bd2895b536,raw,11,catalog,2019-11-01 22:29:32.322458,629881394
13,7b720f69-6562-482e-bcd0-01b5085e9237,raw,13,catalog,2019-11-01 22:30:09.450839,629881394
14,70daf76a-ed57-416c-aed8-17d8daa480d7,raw,14,catalog,2019-11-01 22:31:05.565762,629881394
15,d659ac91-37e2-46bc-9423-d1f3c96865d2,raw,15,main,2019-11-01 22:31:08.333560,629881394
16,a6d20675-a8e0-4fc6-8343-53c95540abf6,raw,16,catalog,2019-11-01 22:31:09.010626,629881394
17,9fb7119e-2e18-43d8-9fe5-6e11fe3ff33c,raw,17,product1,2019-11-01 22:31:10.416231,629881394
18,8731ceb0-5f4c-439d-ba1d-49b827b7bc84,raw,18,catalog,2019-11-01 22:31:43.019527,629881394
19,5ecf46bc-aa9c-4d0a-ab93-e7bc557cd26d,raw,19,catalog,2019-11-01 22:32:01.596163,629881394


In [11]:
print(len(source_df[source_df['user_id'] == check_id]))
source_df[source_df['user_id'] == check_id]

48


,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,80f39ba7-6fe7-4807-b861-0bf310b0d3f8,raw,7,main,2019-11-01 22:28:54.791683,629881394
9,c96cfae1-7f00-49a1-a401-06a5ca509a9c,raw,9,catalog,2019-11-01 22:29:01.049513,629881394
11,897d59ef-4def-473d-b8b8-90bd2895b536,raw,11,catalog,2019-11-01 22:29:32.322458,629881394
13,7b720f69-6562-482e-bcd0-01b5085e9237,raw,13,catalog,2019-11-01 22:30:09.450839,629881394
14,70daf76a-ed57-416c-aed8-17d8daa480d7,raw,14,catalog,2019-11-01 22:31:05.565762,629881394
15,d659ac91-37e2-46bc-9423-d1f3c96865d2,raw,15,main,2019-11-01 22:31:08.333560,629881394
16,a6d20675-a8e0-4fc6-8343-53c95540abf6,raw,16,catalog,2019-11-01 22:31:09.010626,629881394
17,9fb7119e-2e18-43d8-9fe5-6e11fe3ff33c,raw,17,product1,2019-11-01 22:31:10.416231,629881394
18,8731ceb0-5f4c-439d-ba1d-49b827b7bc84,raw,18,catalog,2019-11-01 22:31:43.019527,629881394
19,5ecf46bc-aa9c-4d0a-ab93-e7bc557cd26d,raw,19,catalog,2019-11-01 22:32:01.596163,629881394


User_id = 47562303
Событие ```payment_done``` встречается дважды


In [12]:
check_id = 47562303
print(len(df8[df8['user_id'] == check_id]))
event_time = source_df[(source_df['user_id'] == check_id) & \
    (source_df['event_name'] == 'payment_done')]['event_timestamp'].min()

7


In [13]:
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] <= event_time)]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
4029,4c22eb44-8ebb-4cea-98ae-b3de787268dc,raw,4029,catalog,2019-12-29 16:55:27.897130,47562303
4030,7f3209a7-e4b7-4429-a807-5fd0e7a4c408,raw,4030,cart,2019-12-29 16:55:41.539826,47562303
4031,a8bab30c-ec6a-40ab-b298-22938d728013,raw,4031,delivery_choice,2019-12-29 16:55:44.747151,47562303
4032,c07192ca-168b-4e91-9d36-601e3f7aab33,raw,4032,delivery_courier,2019-12-29 16:55:49.061226,47562303
4033,2cbf4c7a-2aeb-4f06-b85a-9d0111abd287,raw,4033,payment_choice,2019-12-29 16:55:49.201853,47562303
4034,a972d9ff-8d45-4fcc-bc75-4b16989e5a0a,raw,4034,payment_cash,2019-12-29 16:55:52.004250,47562303
4035,078233c6-40d1-4cbe-91cc-81c87414d6a1,raw,4035,payment_done,2019-12-29 16:55:52.350396,47562303


In [14]:
source_df[(source_df['event_name'] == 'payment_done') | \
          (source_df['event_name'] == 'payment_cash') ]\
    .groupby('user_id')['event_name'].nunique().sort_values(ascending=False)

user_id
168688753    2
830731696    2
849341362    2
845778461    2
652347392    2
            ..
355840584    1
358346667    1
363256580    1
365193081    1
999642905    1
Name: event_name, Length: 730, dtype: int64

In [15]:
source_df['event_name'].value_counts()

catalog             14518
main                 5635
lost                 3098
cart                 2842
product2             2172
delivery_choice      1686
product1             1515
payment_choice       1107
delivery_courier      834
payment_done          706
payment_card          565
delivery_pickup       506
payment_cash          197
Name: event_name, dtype: int64

### сдвиг

In [16]:
from src.data_processors_lib.rete import CutPathAfterEvent, CutPathAfterEventParams
params = {
    'cutoff_events': ['payment_done'],
    'cut_shift': 1,
    'min_cjm': 0
}

cut_after = EventsNode(
    CutPathAfterEvent(params=CutPathAfterEventParams(**params)))

graph.add_node(
    node=cut_after,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_after
)

/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_after_event.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut['rw_cumsum'] = df_cut.loc[::-1].groupby([user_col])[time_col].transform(
/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_after_event.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_del['ref'] = df_to_del[eventstream.schema.event_id]


In [17]:
df81 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df81.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [18]:
df81['event_type'].value_counts()

raw    28915
Name: event_type, dtype: int64

In [19]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [20]:
source_df[source_df['event_name'] == 'payment_done']['user_id'].nunique()

653

In [21]:
check_id = 629881394
print('Длина отрезанной траектории - ', len(df81[df81['user_id'] == check_id]))
df81[df81['user_id'] == check_id]

Длина отрезанной траектории -  25


,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,80f39ba7-6fe7-4807-b861-0bf310b0d3f8,raw,7,main,2019-11-01 22:28:54.791683,629881394.0
9,c96cfae1-7f00-49a1-a401-06a5ca509a9c,raw,9,catalog,2019-11-01 22:29:01.049513,629881394.0
11,897d59ef-4def-473d-b8b8-90bd2895b536,raw,11,catalog,2019-11-01 22:29:32.322458,629881394.0
13,7b720f69-6562-482e-bcd0-01b5085e9237,raw,13,catalog,2019-11-01 22:30:09.450839,629881394.0
14,70daf76a-ed57-416c-aed8-17d8daa480d7,raw,14,catalog,2019-11-01 22:31:05.565762,629881394.0
15,d659ac91-37e2-46bc-9423-d1f3c96865d2,raw,15,main,2019-11-01 22:31:08.333560,629881394.0
16,a6d20675-a8e0-4fc6-8343-53c95540abf6,raw,16,catalog,2019-11-01 22:31:09.010626,629881394.0
17,9fb7119e-2e18-43d8-9fe5-6e11fe3ff33c,raw,17,product1,2019-11-01 22:31:10.416231,629881394.0
18,8731ceb0-5f4c-439d-ba1d-49b827b7bc84,raw,18,catalog,2019-11-01 22:31:43.019527,629881394.0
19,5ecf46bc-aa9c-4d0a-ab93-e7bc557cd26d,raw,19,catalog,2019-11-01 22:32:01.596163,629881394.0


In [22]:
# Время, по которому обрезается траектория, здесь задаем конкретное событие payment_cash, для теста надо будет автоматизировать

event_time = source_df[(source_df['user_id'] == check_id) & \
    (source_df['event_name'] == 'payment_cash')]['event_timestamp'].values[0]

In [23]:
print('Длина обрезанной траектории из изначального датасета - ', len(source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] <= event_time)]))
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] <= event_time)]

Длина обрезанной траектории из изначального датасета -  25


,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,80f39ba7-6fe7-4807-b861-0bf310b0d3f8,raw,7,main,2019-11-01 22:28:54.791683,629881394
9,c96cfae1-7f00-49a1-a401-06a5ca509a9c,raw,9,catalog,2019-11-01 22:29:01.049513,629881394
11,897d59ef-4def-473d-b8b8-90bd2895b536,raw,11,catalog,2019-11-01 22:29:32.322458,629881394
13,7b720f69-6562-482e-bcd0-01b5085e9237,raw,13,catalog,2019-11-01 22:30:09.450839,629881394
14,70daf76a-ed57-416c-aed8-17d8daa480d7,raw,14,catalog,2019-11-01 22:31:05.565762,629881394
15,d659ac91-37e2-46bc-9423-d1f3c96865d2,raw,15,main,2019-11-01 22:31:08.333560,629881394
16,a6d20675-a8e0-4fc6-8343-53c95540abf6,raw,16,catalog,2019-11-01 22:31:09.010626,629881394
17,9fb7119e-2e18-43d8-9fe5-6e11fe3ff33c,raw,17,product1,2019-11-01 22:31:10.416231,629881394
18,8731ceb0-5f4c-439d-ba1d-49b827b7bc84,raw,18,catalog,2019-11-01 22:31:43.019527,629881394
19,5ecf46bc-aa9c-4d0a-ab93-e7bc557cd26d,raw,19,catalog,2019-11-01 22:32:01.596163,629881394


In [24]:
print(len(source_df[source_df['user_id'] == check_id]))
source_df[source_df['user_id'] == check_id]

48


,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,80f39ba7-6fe7-4807-b861-0bf310b0d3f8,raw,7,main,2019-11-01 22:28:54.791683,629881394
9,c96cfae1-7f00-49a1-a401-06a5ca509a9c,raw,9,catalog,2019-11-01 22:29:01.049513,629881394
11,897d59ef-4def-473d-b8b8-90bd2895b536,raw,11,catalog,2019-11-01 22:29:32.322458,629881394
13,7b720f69-6562-482e-bcd0-01b5085e9237,raw,13,catalog,2019-11-01 22:30:09.450839,629881394
14,70daf76a-ed57-416c-aed8-17d8daa480d7,raw,14,catalog,2019-11-01 22:31:05.565762,629881394
15,d659ac91-37e2-46bc-9423-d1f3c96865d2,raw,15,main,2019-11-01 22:31:08.333560,629881394
16,a6d20675-a8e0-4fc6-8343-53c95540abf6,raw,16,catalog,2019-11-01 22:31:09.010626,629881394
17,9fb7119e-2e18-43d8-9fe5-6e11fe3ff33c,raw,17,product1,2019-11-01 22:31:10.416231,629881394
18,8731ceb0-5f4c-439d-ba1d-49b827b7bc84,raw,18,catalog,2019-11-01 22:31:43.019527,629881394
19,5ecf46bc-aa9c-4d0a-ab93-e7bc557cd26d,raw,19,catalog,2019-11-01 22:32:01.596163,629881394


User_id = 47562303
Событие встречается дважды


In [25]:
check_id = 47562303
print(len(df81[df81['user_id'] == check_id]))
event_time = source_df[(source_df['user_id'] == check_id) & \
    (source_df['event_name'] == 'payment_done')]['event_timestamp'].min()

6


In [26]:
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] <= event_time)]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
4029,4c22eb44-8ebb-4cea-98ae-b3de787268dc,raw,4029,catalog,2019-12-29 16:55:27.897130,47562303
4030,7f3209a7-e4b7-4429-a807-5fd0e7a4c408,raw,4030,cart,2019-12-29 16:55:41.539826,47562303
4031,a8bab30c-ec6a-40ab-b298-22938d728013,raw,4031,delivery_choice,2019-12-29 16:55:44.747151,47562303
4032,c07192ca-168b-4e91-9d36-601e3f7aab33,raw,4032,delivery_courier,2019-12-29 16:55:49.061226,47562303
4033,2cbf4c7a-2aeb-4f06-b85a-9d0111abd287,raw,4033,payment_choice,2019-12-29 16:55:49.201853,47562303
4034,a972d9ff-8d45-4fcc-bc75-4b16989e5a0a,raw,4034,payment_cash,2019-12-29 16:55:52.004250,47562303
4035,078233c6-40d1-4cbe-91cc-81c87414d6a1,raw,4035,payment_done,2019-12-29 16:55:52.350396,47562303


In [27]:
source_df[(source_df['event_name'] == 'payment_done') | \
          (source_df['event_name'] == 'payment_cash') ]\
    .groupby('user_id')['event_name'].nunique().sort_values(ascending=False)

user_id
168688753    2
830731696    2
849341362    2
845778461    2
652347392    2
            ..
355840584    1
358346667    1
363256580    1
365193081    1
999642905    1
Name: event_name, Length: 730, dtype: int64

In [28]:
source_df['event_name'].value_counts()

catalog             14518
main                 5635
lost                 3098
cart                 2842
product2             2172
delivery_choice      1686
product1             1515
payment_choice       1107
delivery_courier      834
payment_done          706
payment_card          565
delivery_pickup       506
payment_cash          197
Name: event_name, dtype: int64

### min_cjm

In [29]:
from src.data_processors_lib.rete import CutPathAfterEvent, CutPathAfterEventParams
params = {
    'cutoff_events': ['payment_done'],
    'cut_shift': 0,
    'min_cjm': 3
}

cut_after = EventsNode(
    CutPathAfterEvent(params=CutPathAfterEventParams(**params)))

graph.add_node(
    node=cut_after,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_after
)

/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_after_event.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut['rw_cumsum'] = df_cut.loc[::-1].groupby([user_col])[time_col].transform(


In [30]:
df82 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df82.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [31]:
df82['event_type'].value_counts()

raw    31780
Name: event_type, dtype: int64

In [32]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [33]:
source_df[source_df['event_name'] == 'payment_done']['user_id'].nunique()

653

In [34]:
df8.groupby(['user_id']).count()['event_id'].value_counts()

2     443
3     380
4     352
7     321
8     297
5     282
6     265
9     221
10    190
11    137
12    123
13    102
14     97
15     70
18     56
16     56
17     47
19     40
20     39
21     32
22     28
23     26
27     18
24     14
25     12
28     11
26     10
32     10
34      9
31      8
35      8
29      6
45      4
36      4
30      4
33      3
51      3
37      3
46      2
49      2
38      2
75      1
43      1
39      1
47      1
55      1
63      1
57      1
48      1
42      1
44      1
50      1
62      1
73      1
85      1
Name: event_id, dtype: int64

In [35]:
df82.groupby(['user_id']).count()['event_id'].value_counts()

3     380
4     352
7     321
8     297
5     282
6     265
9     221
10    190
11    137
12    123
13    102
14     97
15     70
18     56
16     56
17     47
19     40
20     39
21     32
22     28
23     26
27     18
24     14
25     12
28     11
32     10
26     10
34      9
31      8
35      8
29      6
45      4
36      4
30      4
33      3
51      3
37      3
46      2
49      2
38      2
75      1
43      1
39      1
47      1
55      1
63      1
57      1
48      1
42      1
44      1
50      1
62      1
73      1
85      1
Name: event_id, dtype: int64

### Несколько событий

In [36]:
from src.data_processors_lib.rete import CutPathAfterEvent, CutPathAfterEventParams
params = {
    'cutoff_events': ['payment_done', 'payment_cash', 'delivery_choice'],
    'cut_shift': 0,
    'min_cjm': 0
}

cut_after = EventsNode(
    CutPathAfterEvent(params=CutPathAfterEventParams(**params)))

graph.add_node(
    node=cut_after,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_after
)

/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_after_event.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut['rw_cumsum'] = df_cut.loc[::-1].groupby([user_col])[time_col].transform(
/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_after_event.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_del['ref'] = df_to_del[eventstream.schema.event_id]


In [37]:
df83 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df83.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [38]:
check_id = 168688753
print(len(df83[df83['user_id'] == check_id]))
event_time = source_df[(source_df['user_id'] == check_id) & \
    ((source_df['event_name'] == 'payment_done') | \
    (source_df['event_name'] == 'payment_cash') | \
     (source_df['event_name'] == 'delivery_choice')
     )]['event_timestamp'].min()
event_time

14


Timestamp('2019-12-14 06:33:57.636413')

In [39]:
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] <= event_time)]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
842,76d27684-4d79-43ce-8121-aca4201e83b9,raw,842,catalog,2019-11-16 22:30:05.735414,168688753
843,ceeba921-ed60-4b33-913a-f71dfab1a8d9,raw,843,catalog,2019-11-16 22:30:15.425342,168688753
844,60b2f7f8-2cad-4032-ad5a-837c4b91e1a8,raw,844,catalog,2019-11-16 22:30:40.187585,168688753
2286,9e761508-e35b-4446-9ca0-ed16119de3af,raw,2286,main,2019-12-06 21:40:31.972558,168688753
2833,a911c4b1-7f93-4293-9d40-5473bc32b735,raw,2833,main,2019-12-14 06:32:50.934854,168688753
2834,fbfef058-98ee-4d36-b4b8-4a6694a70a23,raw,2834,catalog,2019-12-14 06:32:56.369665,168688753
2835,eed893b3-2fd0-4fc5-bea2-5dba99793dea,raw,2835,catalog,2019-12-14 06:33:12.102891,168688753
2836,91a68e89-d3dd-4db3-801d-753382905882,raw,2836,catalog,2019-12-14 06:33:28.660860,168688753
2837,cd5ab612-1fa3-4963-9c08-ed84498a901b,raw,2837,product2,2019-12-14 06:33:29.294200,168688753
2838,762c30cb-3469-44cc-89e0-55366d21df93,raw,2838,catalog,2019-12-14 06:33:30.657942,168688753


In [40]:
df83[df83['user_id'] == check_id]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
986,76d27684-4d79-43ce-8121-aca4201e83b9,raw,986,catalog,2019-11-16 22:30:05.735414,168688753.0
987,ceeba921-ed60-4b33-913a-f71dfab1a8d9,raw,987,catalog,2019-11-16 22:30:15.425342,168688753.0
988,60b2f7f8-2cad-4032-ad5a-837c4b91e1a8,raw,988,catalog,2019-11-16 22:30:40.187585,168688753.0
2859,9e761508-e35b-4446-9ca0-ed16119de3af,raw,2859,main,2019-12-06 21:40:31.972558,168688753.0
3535,a911c4b1-7f93-4293-9d40-5473bc32b735,raw,3535,main,2019-12-14 06:32:50.934854,168688753.0
3536,fbfef058-98ee-4d36-b4b8-4a6694a70a23,raw,3536,catalog,2019-12-14 06:32:56.369665,168688753.0
3537,eed893b3-2fd0-4fc5-bea2-5dba99793dea,raw,3537,catalog,2019-12-14 06:33:12.102891,168688753.0
3538,91a68e89-d3dd-4db3-801d-753382905882,raw,3538,catalog,2019-12-14 06:33:28.660860,168688753.0
3539,cd5ab612-1fa3-4963-9c08-ed84498a901b,raw,3539,product2,2019-12-14 06:33:29.294200,168688753.0
3540,762c30cb-3469-44cc-89e0-55366d21df93,raw,3540,catalog,2019-12-14 06:33:30.657942,168688753.0


In [41]:
source_df[(source_df['user_id'] == check_id) ]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
842,76d27684-4d79-43ce-8121-aca4201e83b9,raw,842,catalog,2019-11-16 22:30:05.735414,168688753
843,ceeba921-ed60-4b33-913a-f71dfab1a8d9,raw,843,catalog,2019-11-16 22:30:15.425342,168688753
844,60b2f7f8-2cad-4032-ad5a-837c4b91e1a8,raw,844,catalog,2019-11-16 22:30:40.187585,168688753
2286,9e761508-e35b-4446-9ca0-ed16119de3af,raw,2286,main,2019-12-06 21:40:31.972558,168688753
2833,a911c4b1-7f93-4293-9d40-5473bc32b735,raw,2833,main,2019-12-14 06:32:50.934854,168688753
2834,fbfef058-98ee-4d36-b4b8-4a6694a70a23,raw,2834,catalog,2019-12-14 06:32:56.369665,168688753
2835,eed893b3-2fd0-4fc5-bea2-5dba99793dea,raw,2835,catalog,2019-12-14 06:33:12.102891,168688753
2836,91a68e89-d3dd-4db3-801d-753382905882,raw,2836,catalog,2019-12-14 06:33:28.660860,168688753
2837,cd5ab612-1fa3-4963-9c08-ed84498a901b,raw,2837,product2,2019-12-14 06:33:29.294200,168688753
2838,762c30cb-3469-44cc-89e0-55366d21df93,raw,2838,catalog,2019-12-14 06:33:30.657942,168688753


### Результат

1) [TODO] Ворнинги
2) [DONE] Не работает обрезка
3) [TODO] Медленная работа - оптимизация кода?
4) [DONE] Неправильная работа, если задать несколько событий, обрезка происходит по последнему, а не по первому событию
5) [DONE] протестировать сдвиги и минимальные траектории
6) [TODO] неправильная работа soft_delete и создание ивентстрима (при копировании не пробрасываются старые айдишники у ивентов, а создаются новые, поэтому невозможно применить soft_delete к новому eventstream)

# SplitSessions

In [42]:
from src.data_processors_lib.rete import SplitSessions, SplitSessionsParams
params = {
    'session_cutoff': (1000, 's'),
    'mark_truncated': True,
    'session_col': 'session_id'
}

split_sessions = EventsNode(
    SplitSessions(params=SplitSessionsParams(**params)))

graph.add_node(
    node=split_sessions,
    parents=[graph.root]
)

result = graph.combine(
    node=split_sessions
)


/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/split_sessions.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_del_start.loc[:, 'ref'] = df_to_del_start.loc[:, eventstream.schema.event_id]
/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/split_sessions.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_del_end.loc[:, 'ref'] = df_to_del_end[eventstream.schema.event_id]


In [43]:
df10 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df10.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [44]:
df10.head()

,event_id,event_type,event_index,event_name,event_timestamp,user_id
8,a04e65b5-5ff8-4d59-a739-9a7f0d0b2be8,session_end,8,session_end,2019-11-01 17:59:32.557029,219483890.0
9,17953bd4-a7e5-4e51-8b8d-551e70a6396d,session_start,9,session_start,2019-11-01 21:38:19.283663,964964743.0
10,ebf50876-5ca8-4e0f-9eb0-219f0119c12e,raw,10,catalog,2019-11-01 21:38:19.283663,964964743.0
11,f7aa792d-9ca6-4395-906f-1384b6564513,raw,11,cart,2019-11-01 21:38:36.761221,964964743.0
12,03f2c95e-8880-4b92-afc0-c59f67b35274,raw,12,delivery_choice,2019-11-01 21:38:37.564693,964964743.0


In [45]:
df10['event_type'].value_counts()

raw              35358
session_start     6454
session_end       6452
Name: event_type, dtype: int64

In [46]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [47]:
print(len(df10[df10['user_id'] == 219483890]))
df10[df10['user_id'] == 219483890]

20


,event_id,event_type,event_index,event_name,event_timestamp,user_id
8,a04e65b5-5ff8-4d59-a739-9a7f0d0b2be8,session_end,8,session_end,2019-11-01 17:59:32.557029,219483890.0
3074,9d6d4da6-5d3d-4acf-84ef-d1a8b6a88ce0,session_start,3074,session_start,2019-12-06 16:22:57.484842,219483890.0
3075,805c6238-8394-4d40-83b3-eb92788098b3,raw,3075,main,2019-12-06 16:22:57.484842,219483890.0
3076,c999843d-0bb9-4310-aacf-90f38d8ef845,raw,3076,catalog,2019-12-06 16:23:01.331109,219483890.0
3077,9c781fb1-495d-44a7-bf2e-a22f66738199,raw,3077,catalog,2019-12-06 16:23:48.116617,219483890.0
3078,c53b9014-8b05-4f57-b39a-7d864db1dc0a,session_end,3078,session_end,2019-12-06 16:23:48.116617,219483890.0
6723,67e015e2-61b4-4ff0-9a69-98f20248c655,session_start,6723,session_start,2020-01-06 22:10:13.635011,219483890.0
6724,ecc135b6-90e7-4e4d-aa4f-c23f4cf1faf6,raw,6724,main,2020-01-06 22:10:13.635011,219483890.0
6725,1eafc70e-220a-402a-8e30-fbede35d929f,raw,6725,catalog,2020-01-06 22:10:15.228575,219483890.0
6726,13113d76-04ae-4e6a-bd0b-3a19a148cd76,raw,6726,cart,2020-01-06 22:10:42.309028,219483890.0


In [48]:
print(len(source_df[source_df['user_id'] == 219483890]))
source_df[source_df['user_id'] == 219483890]

17


,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,f7ebd5ed-9760-4227-a33b-01520d8a69fa,raw,0,catalog,2019-11-01 17:59:13.273932,219483890
1,5388c1af-b299-43a0-835d-486413372dca,raw,1,product1,2019-11-01 17:59:28.459271,219483890
2,0fd67b09-1bab-4a6a-a8b1-c5a53bd1758a,raw,2,cart,2019-11-01 17:59:29.502214,219483890
3,798c2959-32bd-43f5-8f6b-3e1739395624,raw,3,catalog,2019-11-01 17:59:32.557029,219483890
2245,805c6238-8394-4d40-83b3-eb92788098b3,raw,2245,main,2019-12-06 16:22:57.484842,219483890
2246,c999843d-0bb9-4310-aacf-90f38d8ef845,raw,2246,catalog,2019-12-06 16:23:01.331109,219483890
2247,9c781fb1-495d-44a7-bf2e-a22f66738199,raw,2247,catalog,2019-12-06 16:23:48.116617,219483890
4836,ecc135b6-90e7-4e4d-aa4f-c23f4cf1faf6,raw,4836,main,2020-01-06 22:10:13.635011,219483890
4837,1eafc70e-220a-402a-8e30-fbede35d929f,raw,4837,catalog,2020-01-06 22:10:15.228575,219483890
4838,13113d76-04ae-4e6a-bd0b-3a19a148cd76,raw,4838,cart,2020-01-06 22:10:42.309028,219483890


In [49]:
source.to_dataframe()

,event_id,event_type,event_index,event_name,event_timestamp,user_id
0,f7ebd5ed-9760-4227-a33b-01520d8a69fa,raw,0,catalog,2019-11-01 17:59:13.273932,219483890
1,5388c1af-b299-43a0-835d-486413372dca,raw,1,product1,2019-11-01 17:59:28.459271,219483890
2,0fd67b09-1bab-4a6a-a8b1-c5a53bd1758a,raw,2,cart,2019-11-01 17:59:29.502214,219483890
3,798c2959-32bd-43f5-8f6b-3e1739395624,raw,3,catalog,2019-11-01 17:59:32.557029,219483890
4,ebf50876-5ca8-4e0f-9eb0-219f0119c12e,raw,4,catalog,2019-11-01 21:38:19.283663,964964743
...,...,...,...,...,...,...
35376,17405fa6-fcec-42b7-90e1-2367fbb8be4a,raw,35376,catalog,2020-04-29 12:47:40.975732,501098384
35377,9f8236d4-e5b2-431e-b578-38cc9b6f071c,raw,35377,catalog,2020-04-29 12:48:01.809577,501098384
35378,bf0c7d44-984e-4c0f-9664-0839ba3ce05e,raw,35378,main,2020-04-29 12:48:01.938488,501098384
35379,9f9bb713-b3b2-481b-9751-8b9f53abaedf,raw,35379,catalog,2020-04-29 12:48:06.595390,501098384


## Результат

1) [DONE] Дубликаты
2) [DONE] Некорректно работает параметр "Mark_truncated" - должны удалиться все события, кроме Start и end_session для сессий обрезанных сначала и end и start_session для сессий обрезанных в конце
3) [TODO] Не добавляется колонка session_id
4) [TODO] более точно проверить разбивку на сессии
5) [TODO] Ворнинги
6) [TODO] Протестить при последовательном добавлении
7) [TODO] неправильная работа soft_delete и создание ивентстрима (при копировании не пробрасываются старые айдишники у ивентов, а создаются новые, поэтому невозможно применить soft_delete к новому eventstream)

Обсудить с ВОлодей
- нужен ли параметр - какие типы событий удалять?


# CutPathBefore

### Описание работы функции

Обрезает eventstream сразу после одного из событий, указанных в списке cutoff_events (первого в траектории). Остается часть траектории, после первого найденного события из списка включительно.

Параметры:
```cut_shift``` - смещение относительно обозначенного в сторону сокращения  (вправо)
```min_cjm``` - минимальное кол-во событий в траектории, которые надо оставить, если в траектории меньшее количество событий, то они будут удалены


### 1 событие

In [50]:
from src.data_processors_lib.rete import CutPathBeforeEvent, CutPathBeforeEventParams
params = {
    'cutoff_events': ['product2'],
    'cut_shift': 0,
    'min_cjm': 0
}

cut_before = EventsNode(
    CutPathBeforeEvent(params=CutPathBeforeEventParams(**params)))

graph.add_node(
    node=cut_before,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_before
)


/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_before_event.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut['num_groups'] = df_cut.groupby([user_col])[time_col].transform(
/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_before_event.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_del['ref'] = df_to_del[eventstream.schema.event_id]


In [51]:
df91 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df91.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [52]:
df91['event_type'].value_counts()

raw    14669
Name: event_type, dtype: int64

In [53]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [54]:
source_df[source_df['event_name'] == 'product2']['user_id'].nunique()

1430

In [55]:
check_id = 629881394
# product2 встречается дважды
print('Новая длина траектории - ', len(source_df[source_df['user_id'] == check_id]))
print('Старая длина траектории - ', len(df91[df91['user_id'] == check_id]))

event_time = source_df[(source_df['user_id'] == check_id) & \
    (source_df['event_name'] == 'product2')]['event_timestamp'].min()

Новая длина траектории -  48
Старая длина траектории -  36


In [56]:
df91[df91['user_id'] == check_id]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
44,a2743119-14b8-498e-abba-fe8f40021b53,raw,44,product2,2019-11-01 22:32:16.496241,629881394.0
45,36a74849-c923-434b-ab1b-c49c813d6b14,raw,45,catalog,2019-11-01 22:33:17.682468,629881394.0
46,b9e72bd3-b08d-4107-bb3d-c85cb02ccead,raw,46,product1,2019-11-01 22:33:19.961115,629881394.0
47,7c0023c7-cc16-4089-9f80-f42077e34d98,raw,47,catalog,2019-11-01 22:33:35.460345,629881394.0
50,aaf77e10-0adf-4b3f-9f41-4c7cd66b5b4e,raw,50,catalog,2019-11-01 22:34:02.301293,629881394.0
53,83db7c69-1bfe-4cb0-a40a-bffc5d24159d,raw,53,product2,2019-11-01 22:34:32.362386,629881394.0
55,ad448920-16e6-4a36-975a-abec4b3e0e6c,raw,55,catalog,2019-11-01 22:35:01.462515,629881394.0
57,a9e4d43c-5a97-442b-88c0-9afa15380323,raw,57,product2,2019-11-01 22:35:33.142711,629881394.0
60,ba54d47e-8105-4e04-a055-ef473719a441,raw,60,cart,2019-11-01 22:35:50.437706,629881394.0
62,e980bc98-8cee-4ca5-a062-887f1be61956,raw,62,delivery_choice,2019-11-01 22:35:57.649549,629881394.0


In [57]:
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] >= event_time)]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
22,a2743119-14b8-498e-abba-fe8f40021b53,raw,22,product2,2019-11-01 22:32:16.496241,629881394
23,36a74849-c923-434b-ab1b-c49c813d6b14,raw,23,catalog,2019-11-01 22:33:17.682468,629881394
24,b9e72bd3-b08d-4107-bb3d-c85cb02ccead,raw,24,product1,2019-11-01 22:33:19.961115,629881394
25,7c0023c7-cc16-4089-9f80-f42077e34d98,raw,25,catalog,2019-11-01 22:33:35.460345,629881394
27,aaf77e10-0adf-4b3f-9f41-4c7cd66b5b4e,raw,27,catalog,2019-11-01 22:34:02.301293,629881394
29,83db7c69-1bfe-4cb0-a40a-bffc5d24159d,raw,29,product2,2019-11-01 22:34:32.362386,629881394
31,ad448920-16e6-4a36-975a-abec4b3e0e6c,raw,31,catalog,2019-11-01 22:35:01.462515,629881394
33,a9e4d43c-5a97-442b-88c0-9afa15380323,raw,33,product2,2019-11-01 22:35:33.142711,629881394
36,ba54d47e-8105-4e04-a055-ef473719a441,raw,36,cart,2019-11-01 22:35:50.437706,629881394
38,e980bc98-8cee-4ca5-a062-887f1be61956,raw,38,delivery_choice,2019-11-01 22:35:57.649549,629881394


In [58]:
print(len(source_df[source_df['user_id'] == check_id]))
source_df[source_df['user_id'] == check_id]

48


,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,80f39ba7-6fe7-4807-b861-0bf310b0d3f8,raw,7,main,2019-11-01 22:28:54.791683,629881394
9,c96cfae1-7f00-49a1-a401-06a5ca509a9c,raw,9,catalog,2019-11-01 22:29:01.049513,629881394
11,897d59ef-4def-473d-b8b8-90bd2895b536,raw,11,catalog,2019-11-01 22:29:32.322458,629881394
13,7b720f69-6562-482e-bcd0-01b5085e9237,raw,13,catalog,2019-11-01 22:30:09.450839,629881394
14,70daf76a-ed57-416c-aed8-17d8daa480d7,raw,14,catalog,2019-11-01 22:31:05.565762,629881394
15,d659ac91-37e2-46bc-9423-d1f3c96865d2,raw,15,main,2019-11-01 22:31:08.333560,629881394
16,a6d20675-a8e0-4fc6-8343-53c95540abf6,raw,16,catalog,2019-11-01 22:31:09.010626,629881394
17,9fb7119e-2e18-43d8-9fe5-6e11fe3ff33c,raw,17,product1,2019-11-01 22:31:10.416231,629881394
18,8731ceb0-5f4c-439d-ba1d-49b827b7bc84,raw,18,catalog,2019-11-01 22:31:43.019527,629881394
19,5ecf46bc-aa9c-4d0a-ab93-e7bc557cd26d,raw,19,catalog,2019-11-01 22:32:01.596163,629881394


### сдвиг

In [59]:
from src.data_processors_lib.rete import CutPathBeforeEvent, CutPathBeforeEventParams
params = {
    'cutoff_events': ['product2'],
    'cut_shift': 1,
    'min_cjm': 0
}

cut_before = EventsNode(
    CutPathBeforeEvent(params=CutPathBeforeEventParams(**params)))

graph.add_node(
    node=cut_before,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_before
)


/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_before_event.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut['num_groups'] = df_cut.groupby([user_col])[time_col].transform(
/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_before_event.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_del['ref'] = df_to_del[eventstream.schema.event_id]


In [60]:
df92 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df92.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [61]:
df92['event_type'].value_counts()

raw    13239
Name: event_type, dtype: int64

In [62]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [63]:
source_df[source_df['event_name'] == 'product2']['user_id'].nunique()

1430

In [64]:
check_id = 629881394
print(f'Длина отрезанной траектории для пользователя {check_id} - ',
      len(df92[df92['user_id'] == check_id]))
df92[df92['user_id'] == check_id]

Длина отрезанной траектории для пользователя 629881394 -  35


,event_id,event_type,event_index,event_name,event_timestamp,user_id
46,36a74849-c923-434b-ab1b-c49c813d6b14,raw,46,catalog,2019-11-01 22:33:17.682468,629881394.0
47,b9e72bd3-b08d-4107-bb3d-c85cb02ccead,raw,47,product1,2019-11-01 22:33:19.961115,629881394.0
48,7c0023c7-cc16-4089-9f80-f42077e34d98,raw,48,catalog,2019-11-01 22:33:35.460345,629881394.0
51,aaf77e10-0adf-4b3f-9f41-4c7cd66b5b4e,raw,51,catalog,2019-11-01 22:34:02.301293,629881394.0
54,83db7c69-1bfe-4cb0-a40a-bffc5d24159d,raw,54,product2,2019-11-01 22:34:32.362386,629881394.0
57,ad448920-16e6-4a36-975a-abec4b3e0e6c,raw,57,catalog,2019-11-01 22:35:01.462515,629881394.0
59,a9e4d43c-5a97-442b-88c0-9afa15380323,raw,59,product2,2019-11-01 22:35:33.142711,629881394.0
62,ba54d47e-8105-4e04-a055-ef473719a441,raw,62,cart,2019-11-01 22:35:50.437706,629881394.0
64,e980bc98-8cee-4ca5-a062-887f1be61956,raw,64,delivery_choice,2019-11-01 22:35:57.649549,629881394.0
65,0675c4c2-7948-4f09-8d8c-40a70d2e4e09,raw,65,delivery_courier,2019-11-01 22:36:02.009271,629881394.0


In [65]:
# Время, по которому обрезается траектория, здесь задаем конкретное событие payment_cash, для теста надо будет автоматизировать

event_time = source_df[(source_df['user_id'] == check_id) & \
    (source_df['event_name'] == 'product2')]['event_timestamp'].min()

In [66]:
print(f'Длина обрезанной траектории из source_df для пользователя  {check_id} - ', len(source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] > event_time)]))
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] > event_time)]

Длина обрезанной траектории из source_df для пользователя  629881394 -  35


,event_id,event_type,event_index,event_name,event_timestamp,user_id
23,36a74849-c923-434b-ab1b-c49c813d6b14,raw,23,catalog,2019-11-01 22:33:17.682468,629881394
24,b9e72bd3-b08d-4107-bb3d-c85cb02ccead,raw,24,product1,2019-11-01 22:33:19.961115,629881394
25,7c0023c7-cc16-4089-9f80-f42077e34d98,raw,25,catalog,2019-11-01 22:33:35.460345,629881394
27,aaf77e10-0adf-4b3f-9f41-4c7cd66b5b4e,raw,27,catalog,2019-11-01 22:34:02.301293,629881394
29,83db7c69-1bfe-4cb0-a40a-bffc5d24159d,raw,29,product2,2019-11-01 22:34:32.362386,629881394
31,ad448920-16e6-4a36-975a-abec4b3e0e6c,raw,31,catalog,2019-11-01 22:35:01.462515,629881394
33,a9e4d43c-5a97-442b-88c0-9afa15380323,raw,33,product2,2019-11-01 22:35:33.142711,629881394
36,ba54d47e-8105-4e04-a055-ef473719a441,raw,36,cart,2019-11-01 22:35:50.437706,629881394
38,e980bc98-8cee-4ca5-a062-887f1be61956,raw,38,delivery_choice,2019-11-01 22:35:57.649549,629881394
39,0675c4c2-7948-4f09-8d8c-40a70d2e4e09,raw,39,delivery_courier,2019-11-01 22:36:02.009271,629881394


In [67]:
print(len(source_df[source_df['user_id'] == check_id]))
source_df[source_df['user_id'] == check_id]

48


,event_id,event_type,event_index,event_name,event_timestamp,user_id
7,80f39ba7-6fe7-4807-b861-0bf310b0d3f8,raw,7,main,2019-11-01 22:28:54.791683,629881394
9,c96cfae1-7f00-49a1-a401-06a5ca509a9c,raw,9,catalog,2019-11-01 22:29:01.049513,629881394
11,897d59ef-4def-473d-b8b8-90bd2895b536,raw,11,catalog,2019-11-01 22:29:32.322458,629881394
13,7b720f69-6562-482e-bcd0-01b5085e9237,raw,13,catalog,2019-11-01 22:30:09.450839,629881394
14,70daf76a-ed57-416c-aed8-17d8daa480d7,raw,14,catalog,2019-11-01 22:31:05.565762,629881394
15,d659ac91-37e2-46bc-9423-d1f3c96865d2,raw,15,main,2019-11-01 22:31:08.333560,629881394
16,a6d20675-a8e0-4fc6-8343-53c95540abf6,raw,16,catalog,2019-11-01 22:31:09.010626,629881394
17,9fb7119e-2e18-43d8-9fe5-6e11fe3ff33c,raw,17,product1,2019-11-01 22:31:10.416231,629881394
18,8731ceb0-5f4c-439d-ba1d-49b827b7bc84,raw,18,catalog,2019-11-01 22:31:43.019527,629881394
19,5ecf46bc-aa9c-4d0a-ab93-e7bc557cd26d,raw,19,catalog,2019-11-01 22:32:01.596163,629881394


### min_cjm

In [68]:
from src.data_processors_lib.rete import CutPathBeforeEvent, CutPathBeforeEventParams
params = {
    'cutoff_events': ['product2'],
    'cut_shift': 0,
    'min_cjm': 3
}

cut_before = EventsNode(
    CutPathBeforeEvent(params=CutPathBeforeEventParams(**params)))

graph.add_node(
    node=cut_before,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_before
)


/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_before_event.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut['num_groups'] = df_cut.groupby([user_col])[time_col].transform(


In [69]:
df93 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df93.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

In [70]:
df93['event_type'].value_counts()

raw    14487
Name: event_type, dtype: int64

In [71]:
source_df['event_type'].value_counts()

raw    35381
Name: event_type, dtype: int64

In [72]:
df91.groupby(['user_id']).count()['event_id'].value_counts().sort_index()

2      91
3     171
4     128
5     106
6     124
7     137
8      97
9      57
10     55
11     63
12     50
13     38
14     24
15     31
16     33
17     23
18     22
19     23
20     18
21     13
22     13
23      6
24      8
25      5
26      5
27     13
28      7
29      5
30      6
31      3
32      5
33      4
34      3
35      4
36      2
37      2
38      2
39      3
40      1
41      4
42      2
44      2
46      1
48      3
49      1
50      1
52      1
55      1
60      1
62      1
67      1
68      3
69      1
71      1
73      1
77      1
81      1
84      1
88      1
Name: event_id, dtype: int64

In [80]:
df93.groupby(['user_id']).count()['event_id'].value_counts().sort_index()

3     171
4     128
5     106
6     124
7     137
8      97
9      57
10     55
11     63
12     50
13     38
14     24
15     31
16     33
17     23
18     22
19     23
20     18
21     13
22     13
23      6
24      8
25      5
26      5
27     13
28      7
29      5
30      6
31      3
32      5
33      4
34      3
35      4
36      2
37      2
38      2
39      3
40      1
41      4
42      2
44      2
46      1
48      3
49      1
50      1
52      1
55      1
60      1
62      1
67      1
68      3
69      1
71      1
73      1
77      1
81      1
84      1
88      1
Name: event_id, dtype: int64

### Несколько событий

In [81]:
from src.data_processors_lib.rete import CutPathBeforeEvent, CutPathBeforeEventParams
params = {
    'cutoff_events': ['product2', 'product1'],
    'cut_shift': 0,
    'min_cjm': 0
}

cut_before = EventsNode(
    CutPathBeforeEvent(params=CutPathBeforeEventParams(**params)))

graph.add_node(
    node=cut_before,
    parents=[graph.root]
)

result = graph.combine(
    node=cut_before
)


/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_before_event.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cut['num_groups'] = df_cut.groupby([user_col])[time_col].transform(
/Users/Daria/Dev/Retentioneering/retentioneering-tools-new-arch/examples/../src/data_processors_lib/rete/cut_path_before_event.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_del['ref'] = df_to_del[eventstream.schema.event_id]


In [83]:
df94 = result.to_dataframe()
print(source_df.duplicated(['event_name', 'event_timestamp', 'user_id']).sum())
df94.duplicated(['event_name', 'event_timestamp', 'user_id']).sum()

0


0

0


0

In [86]:
check_id = 168688753
print(len(df94[df94['user_id'] == check_id]))
event_time = source_df[(source_df['user_id'] == check_id) & \
    ((source_df['event_name'] == 'product1') | \
    (source_df['event_name'] == 'product2')
     )]['event_timestamp'].min()
event_time

37


Timestamp('2019-12-14 06:33:29.294200')

In [87]:
source_df[(source_df['user_id'] == check_id) &\
          (source_df['event_timestamp'] >= event_time)]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
2837,cd5ab612-1fa3-4963-9c08-ed84498a901b,raw,2837,product2,2019-12-14 06:33:29.294200,168688753
2838,762c30cb-3469-44cc-89e0-55366d21df93,raw,2838,catalog,2019-12-14 06:33:30.657942,168688753
2839,370c8e37-253a-4d3c-9fef-d557a86b0ec2,raw,2839,catalog,2019-12-14 06:33:38.687904,168688753
2840,b0fa96f9-8785-4c47-85fe-d0ef55dd634e,raw,2840,product1,2019-12-14 06:33:54.105418,168688753
2841,86d2eba2-f69c-42c5-a719-cf6b157445bc,raw,2841,cart,2019-12-14 06:33:57.608245,168688753
2842,1e4e6e0e-10f9-4123-ac18-5637a8b1d08d,raw,2842,delivery_choice,2019-12-14 06:33:57.636413,168688753
2843,7b03dfb9-6846-4520-b5f2-0214f2a97c85,raw,2843,catalog,2019-12-14 06:34:04.620721,168688753
2844,7ecdf64d-727e-4b44-a46e-6357a295096c,raw,2844,catalog,2019-12-14 06:34:09.161217,168688753
2845,504a50d5-1f62-4855-8d85-e78b39c0255d,raw,2845,catalog,2019-12-14 06:34:12.994666,168688753
2846,0e448096-c477-4094-ab96-21bcef15c4c4,raw,2846,catalog,2019-12-14 06:34:37.431860,168688753


In [85]:
df94[df94['user_id'] == check_id]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
4241,cd5ab612-1fa3-4963-9c08-ed84498a901b,raw,4241,product2,2019-12-14 06:33:29.294200,168688753.0
4242,762c30cb-3469-44cc-89e0-55366d21df93,raw,4242,catalog,2019-12-14 06:33:30.657942,168688753.0
4243,370c8e37-253a-4d3c-9fef-d557a86b0ec2,raw,4243,catalog,2019-12-14 06:33:38.687904,168688753.0
4244,b0fa96f9-8785-4c47-85fe-d0ef55dd634e,raw,4244,product1,2019-12-14 06:33:54.105418,168688753.0
4245,86d2eba2-f69c-42c5-a719-cf6b157445bc,raw,4245,cart,2019-12-14 06:33:57.608245,168688753.0
4246,1e4e6e0e-10f9-4123-ac18-5637a8b1d08d,raw,4246,delivery_choice,2019-12-14 06:33:57.636413,168688753.0
4247,7b03dfb9-6846-4520-b5f2-0214f2a97c85,raw,4247,catalog,2019-12-14 06:34:04.620721,168688753.0
4248,7ecdf64d-727e-4b44-a46e-6357a295096c,raw,4248,catalog,2019-12-14 06:34:09.161217,168688753.0
4249,504a50d5-1f62-4855-8d85-e78b39c0255d,raw,4249,catalog,2019-12-14 06:34:12.994666,168688753.0
4250,0e448096-c477-4094-ab96-21bcef15c4c4,raw,4250,catalog,2019-12-14 06:34:37.431860,168688753.0


In [79]:
source_df[(source_df['user_id'] == check_id) ]

,event_id,event_type,event_index,event_name,event_timestamp,user_id
842,76d27684-4d79-43ce-8121-aca4201e83b9,raw,842,catalog,2019-11-16 22:30:05.735414,168688753
843,ceeba921-ed60-4b33-913a-f71dfab1a8d9,raw,843,catalog,2019-11-16 22:30:15.425342,168688753
844,60b2f7f8-2cad-4032-ad5a-837c4b91e1a8,raw,844,catalog,2019-11-16 22:30:40.187585,168688753
2286,9e761508-e35b-4446-9ca0-ed16119de3af,raw,2286,main,2019-12-06 21:40:31.972558,168688753
2833,a911c4b1-7f93-4293-9d40-5473bc32b735,raw,2833,main,2019-12-14 06:32:50.934854,168688753
2834,fbfef058-98ee-4d36-b4b8-4a6694a70a23,raw,2834,catalog,2019-12-14 06:32:56.369665,168688753
2835,eed893b3-2fd0-4fc5-bea2-5dba99793dea,raw,2835,catalog,2019-12-14 06:33:12.102891,168688753
2836,91a68e89-d3dd-4db3-801d-753382905882,raw,2836,catalog,2019-12-14 06:33:28.660860,168688753
2837,cd5ab612-1fa3-4963-9c08-ed84498a901b,raw,2837,product2,2019-12-14 06:33:29.294200,168688753
2838,762c30cb-3469-44cc-89e0-55366d21df93,raw,2838,catalog,2019-12-14 06:33:30.657942,168688753


### Результат

1) [TODO] Ворнинги
2) [DONE] Не работает обрезка
3) [TODO] Медленная работа - оптимизация кода?
4) [DONE] Неправильная работа, если задать несколько событий, обрезка происходит по последнему, а не по первому событию
5) [DONE] протестировать сдвиги и минимальные траектории
6) [TODO] неправильная работа soft_delete и создание eventstream (при копировании не пробрасываются старые айдишники у ивентов, а создаются новые, поэтому невозможно применить soft_delete к новому eventstream)